In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import sys
import os

In [3]:
sys.path.append(os.path.dirname(os.path.abspath('.')))

## Data Source

Streaming data downloaded from this [source](https://hcad.org/pdata/pdata-property-downloads.html).
  
[Ownership](https://download.hcad.org/data/CAMA/2021/Real_acct_ownership_history.zip)
Current ownership information.

In [4]:
import zipfile
import pandas as pd

In [5]:
with zipfile.ZipFile('../data/Real_acct_ownership_history.zip') as f:
    with f.open('ownership_history.txt') as ft:
        df = pd.read_csv(ft, sep='\t', encoding='Latin')

In [6]:
df

,acct,purchase_date,name,site_address
0,10010000013,01/02/1988,CITY OF HOUSTON,0 COMMERCE ST
1,10020000001,10/31/2012,CURRENT OWNER,907 COMMERCE ST
2,10020000003,01/02/1988,MILBY CHARLES FAMILY PTNSH,0 COMMERCE ST
3,10020000004,01/02/1996,FEAGIN MICHAEL RYAN TRUST,0 COMMERCE ST
4,10020000013,12/29/1999,BUFFALO BAYOU PARTNERSHIP,921 COMMERCE ST
...,...,...,...,...
1506718,9700000000235,12/08/1998,PCI NITROGEN LLC,2000 DAVISON RD
1506719,9700000000237,01/01/2003,CENTERPOINT ENERGY HOU ELE,2012 MILLER CUTRD
1506720,9700000000239,01/01/2006,INEOS USA LLC,1515 MILLER CUTRD
1506721,9700000000277,01/01/2002,AKZO NOBEL POLYMER CHEMICALS LLC,12900 BAYPARK RD


In [7]:
from time import sleep, time
import datetime
from json import dumps
from kafka import KafkaProducer

In [8]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [9]:
import random
import time

In [ ]:
start1 = start = time.time()
c = 0
while True:
    for i, row in df.iterrows():
        drow = {
            "post_uuid": row.acct, 
            "post_author_uuid": row['name'],
            "post_name": row.site_address,
            "post_text": f"{row['name'] } just purchased property on {row.site_address} with account number {row.acct}",
            "post_url_slug": row.acct, 
            "post_scope": "global" if random.randint(1, 10) > 7 else "group",
            "post_media": [{"media/1": "property docs"}] if random.randint(1, 10) > 7 else []
        }
        producer.send('post-stream', value=drow)
        c+=1
        sleep(0.0001)
        if time.time() - start1 > 1:
            start1 = time.time()
            speed = c/(start1-start)
            speed = f"{(speed):0.2f}"
            m = f"-- Producing Speed: { speed } Posts Per Second | total Posts: {c}                         "
            print(m, end='\r')